In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
print(os.getcwd())
os.chdir('/content/drive/MyDrive/')

/content


###Reminder to remove LOG from modules.py and functions.py. Later on might have to remove it from every file (or reinstate log)

In [3]:
import numpy as np

class PropagationPath:
    def __init__(self):
        self.points = []
        self.interaction_types = []   # 0 - initial transmitter (radio antenna)   1 - final point (user)   2 - specular reflection   3 - diffraction around the edge
        self.path_gain_db = 0   # dB = 10 * log10(I/I0)   -20 dB = 100 times weaker   -90 dB = 1000000000 times weaker
        self.hash = 0   # hashed set of interactions (objects / surfaces)

offset = 0
def read_int():
    global offset, bytes
    offset += 4
    return bytes[offset-4:offset].copy().view(np.int32)[0]
def read_uint():
    global offset, bytes
    offset += 4
    return bytes[offset-4:offset].copy().view(np.uint32)[0]
def read_float():
    global offset, bytes
    offset += 4
    return bytes[offset-4:offset].copy().view(np.float32)[0]

def read_file(filename: str):
    global offset, bytes
    offset = 0
    bytes = np.fromfile(filename, dtype=np.uint8)

    transmitter_count = read_int()
    paths = []
    for tx in range(transmitter_count):
        receiver_count = read_int()
        paths.append([])
        for rx in range(receiver_count):
            path_count = read_int()
            paths[tx].append([])
            for p in range(path_count):
                path = PropagationPath()
                point_count = read_int()
                for _ in range(point_count):
                    x = read_float()
                    y = read_float()
                    z = read_float()
                    path.points.append((x, y, z))
                for _ in range(point_count):
                    interaction = read_int()
                    path.interaction_types.append(interaction)

                path.path_gain_db = read_float()
                path.hash = read_uint()
                paths[tx][rx].append(path)
    return paths

if __name__ == "__main__":
    path_file = 'eu_city_2x2_macro_306.bin'
    paths = read_file(path_file)


In [4]:
print(paths[0])
print(len(paths))
print(len(paths[0]))
print(len(paths[1]))

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [],

In [81]:
#Find first non-zero path
for i in range(len(paths[0])):
  if len(paths[0][i])!=0:
    print(i)
    break

7182


### Only using 1 data sample right now. Trying to get it to work before looking at bigger batch sizes

In [6]:
print(paths[0][7182][1].points)
print(paths[0][7182][1].interaction_types)
print(paths[0][7182][1].path_gain_db)
print(paths[0][7182][1].hash)

[(158.49117, -32.520683, 53.320007), (-342.37683, -341.62332, 56.24), (-348.43402, -345.3433, 56.24), (-360.13348, -351.58405, 43.58853), (-371.9333, -369.0092, 23.50962)]
[0, 3, 3, 2, 1]
-148.83165
837731744


In [7]:
!pip install ttools

  Preparing metadata (setup.py) ... done
  Created wheel for ttools: filename=ttools-0.1-py3-none-any.whl size=1205 sha256=ceffe0f6e8c63645dadb773a020a5aabdccc491aaed4a65814436579b5ff2d58
  Stored in directory: /root/.cache/pip/wheels/e2/ed/d7/89a4523eca26412a0092b3d5395284fd0f3e0cceaedf8176cf
Successfully built ttools


In [11]:
import torch
import torch.nn as nn
#import sys
#sys.path.insert(1, '/content/drive/MyDrive/sbmcFiles/')
from sbmcFiles import modules as ops

In [ ]:
a=paths[0][7182][1]#4dim

In [45]:
data_arrays = np.array(a.points)  # Convert the list of tuples to a NumPy array
data_tensor = torch.tensor(data_arrays)  # Convert the NumPy array to a PyTorch tensor
data_indices = torch.tensor(a.interaction_types, dtype=torch.long)  # Convert the list of integers to a PyTorch tensor
float_gain = torch.tensor(a.path_gain_db, dtype=torch.float32)  # Convert the float to a PyTorch tensor
float_hash= torch.tensor(a.hash, dtype=torch.long)

In [46]:
data_tensor.shape

torch.Size([5, 3])

In [63]:
data_tensor = data_tensor.unsqueeze(0)# add batch dim
print(data_tensor)

tensor([[[ 158.4912,  -32.5207,   53.3200],
         [-342.3768, -341.6233,   56.2400],
         [-348.4340, -345.3433,   56.2400],
         [-360.1335, -351.5840,   43.5885],
         [-371.9333, -369.0092,   23.5096]]])


In [64]:
print(data_tensor.shape)

torch.Size([1, 5, 3])


In [66]:

embedding=ops.ConvChain(len(paths[0][7182][1].points),64,width=64,depth=3,ksize=1, pad=False)

embedding=ops.ConvChain(1,64,width=64,depth=3,ksize=1, pad=False)

In [56]:
print(len(a.points))
print(embedding)

5
ConvChain(
  (layer_0): _ConvBNRelu(
    (layer): Sequential(
      (0): Conv2d(4, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): ReLU()
    )
  )
  (layer_1): _ConvBNRelu(
    (layer): Sequential(
      (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): ReLU()
    )
  )
  (prediction): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
)


In [82]:
#combined_tensor = torch.cat((data_tensor, data_indices.unsqueeze(1), float_gain.unsqueeze(1),float_hash.unsqueeze(1)), dim=1)
#stacked_tensor = torch.stack((data_tensor, data_indices.unsqueeze(-1), float_gain.unsqueeze(-1), float_hash.unsqueeze(-1)), dim=-1)

In [69]:
print(embedding(data_tensor).shape)
flat=embedding(data_tensor)


torch.Size([64, 5, 3])


In [80]:
#print(flat)

In [70]:
propagation = ops.Autoencoder(64, 64, num_levels=3,
                increase_factor=2.0, num_convs=3, width=64, ksize=3,
                output_type="leaky_relu", pooling="max")

In [86]:
flat.mean(1)
print(flat.shape)
test=propagation(flat.mean(1).unsqueeze(0))

torch.Size([64, 5, 3])


RuntimeError: ignored